# Guitar Lead/Rhythm Separation Model Training

This notebook trains a mel_band_roformer model to separate lead guitar from rhythm guitar.

**Before running:**
1. Upload `guitar_training_package.zip` to your Google Drive root folder
2. Make sure you're using a GPU runtime (Runtime > Change runtime type > T4 GPU)

**Estimated time:** 8-12 hours for full training on T4

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Unzip training package
!unzip -q /content/drive/MyDrive/guitar_training_package.zip -d /content/training
!ls /content/training

In [ ]:
# Install dependencies
!pip install -q ml_collections einops rotary_embedding_torch beartype loralib wandb auraloss torch_audiomentations audiomentations pedalboard

In [ ]:
# Update config for full training (100 epochs, 500 steps)
import yaml

config_path = '/content/training/configs/config_guitar_lead_rhythm.yaml'
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# Set for full training
config['training']['num_epochs'] = 100
config['training']['num_steps'] = 500
config['training']['use_amp'] = True  # Enable AMP for GPU
config['training']['ema_momentum'] = 0.999  # Re-enable EMA

with open(config_path, 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

print("Config updated for full training:")
print(f"  Epochs: {config['training']['num_epochs']}")
print(f"  Steps per epoch: {config['training']['num_steps']}")
print(f"  AMP: {config['training']['use_amp']}")

In [ ]:
# Start training
%cd /content/training

!python train.py \
    --model_type mel_band_roformer \
    --config_path configs/config_guitar_lead_rhythm.yaml \
    --data_path dataset/training \
    --results_path /content/drive/MyDrive/guitar_model_results \
    --dataset_type 1 \
    --device_ids 0 \
    --num_workers 2 \
    --pin_memory

## Training Complete!

Your trained model will be saved to `Google Drive/guitar_model_results/`

The model file will be named something like `model_mel_band_roformer_ep_XXX_sdr_XX.XXXX.ckpt`

In [ ]:
# List trained models
!ls -la /content/drive/MyDrive/guitar_model_results/*.ckpt 2>/dev/null || echo "No models saved yet"